In [1]:
import os
from pydub import AudioSegment
import pandas
import numpy
import requests
from IPython.display import clear_output

In [2]:
birds = pandas.read_csv('birds_dataset.csv', index_col=0)

In [3]:
birds.head()

,index,file_id,genus,species,name,download_url,file_name,duration,other_species_in_recording
0,8,452850,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/452850/download,"XC452850-Cinereous Tinamou, song, 190117-023.mp3",53,"Poecilotriccus latirostris, Pheugopedius genib..."
1,9,446610,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/446610/download,XC446610-Crypturellus cinereus - DaRec_2018052...,32,NaN
2,10,424163,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/424163/download,XC424163-Crypturellus cinereus. RO. ME- Felipe...,36,"Sclerurus obscurior, Amazona farinosa"
3,11,333322,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/333322/download,XC333322-CRYCIN13.mp3,51,"Thamnophilus aethiops, Hypocnemis ochrogyna, M..."
4,12,329091,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/329091/download,XC329091-Crypturellus cinereus(song)XapuriAC23...,37,NaN


In [4]:
genus_vs_file_path = pandas.DataFrame(columns=['file_id', 'genus', 'file_path'])

In [5]:
def convert(src_file_name, dst_file_name, dest_folder='audio\\'):
    dst_file_name = dst_file_name + '.wav'
    sound = AudioSegment.from_mp3(src_file_name)[:20000]
    sound.export(dest_folder + dst_file_name, format="wav")
    print('converted ' + dst_file_name)
    return dest_folder + dst_file_name

In [6]:
error_list = []
for index, row in birds.iterrows():
    try:
        clear_output(wait=True)
        r = requests.get(row.download_url, allow_redirects=True)
        with open(str(row.file_id)+'.mp3','wb') as op:
          op.write(r.content)
        dst_file_path = convert(str(row.file_id)+'.mp3', str(row.file_id))
        os.remove(str(row.file_id)+'.mp3')
        genus_vs_file_path = genus_vs_file_path.append({
                'file_id': row['file_id'],
                'genus': row['genus'],
                'file_path': dst_file_path
            }, ignore_index=True)
        print('Download and conversion in finished for {} of {} files'.format(index+1, birds.shape[0]))
    except Exception as e:
        error_list.append(row.download_url)
        print(e)

Download and conversion in finished for 6208 of 6210 files


In [7]:
genus_vs_file_path.to_csv('genus_vs_file_path.csv')